In [8]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [9]:
# !pip3 install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [10]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [11]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [12]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [13]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [14]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:22 Time:  0:00:22
100% (40 of 40) |########################| Elapsed Time: 0:00:21 Time:  0:00:21


In [15]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:22 Time:  0:00:22
100% (40 of 40) |########################| Elapsed Time: 0:00:20 Time:  0:00:20


In [16]:
nbatest.shape

(786, 3)

In [17]:
nfltest.shape

(942, 3)

In [20]:
nfltest.head()

,subreddit,title,selftext
t3_ei5he6,nfl,Talko Tuesday,"Welcome to today's open thread, where /r/nfl u..."
t3_ehwkan,nfl,/r/NFL Best Of 2019 Nomination Thread,"#2019 has been a great year for football, so l..."
t3_ei32ez,nfl,[Silver] The Ron Rivera deal is done. He has r...,
t3_ei2x87,nfl,[Schefter] Giants are expected to interview fo...,
t3_ei7htz,nfl,[Schefter]John Dorsey is out as the Browns’ GM...,


##### These scrape_add functions add to already built csvs

In [ ]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [21]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [22]:
# drop column

nfltest = nfltest.drop(columns='selftext')
nbatest = nbatest.drop(columns='selftext')

In [23]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [25]:
train.tail()

,subreddit,title
t3_egpz42,nba,NBA Decade Awards of Reddittors - What's your opinion?
t3_egogrx,nba,Embiid leads the Sixers in FT%
t3_egmt2s,nba,Six Heat players have scored 25 or more points multiple times this season
t3_eglnq6,nba,The Tale of 2011 Dirk
t3_ei88id,nba,Is Giannis pulling away from everyone else in the MVP race?


##### Tokenize (grab only word characters)

In [32]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [33]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [34]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [36]:
train.head()

,subreddit,title
t3_ei5he6,nfl,talko tuesday
t3_ehwkan,nfl,r nfl best of 2019 nomination thread
t3_ei32ez,nfl,silver the ron rivera deal is done he has reached an agreement to become the new coach of the redskins according to a source familiar with negotiations aroundthenfl nflnetwork
t3_ei2x87,nfl,schefter giants are expected to interview former packers head coach mike mccarthy this weekend per source
t3_ei7htz,nfl,schefter john dorsey is out as the browns gm per source two sides couldn t come to an agreement on a future re structure of the organization


### Train test split and converting series to list of strings then to array

In [37]:
X = train[['title']]
y = train['subreddit']

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [44]:
# baseline is

y.value_counts(normalize=True)

nfl    0.545139
nba    0.454861
Name: subreddit, dtype: float64

In [45]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [46]:
len(clean_train_data)

1296

In [47]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [48]:
len(clean_test_data)

432

### Count Vectorizer

In [50]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [51]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [52]:
# convert to array

train_data_features = train_data_features.toarray()

In [53]:
train_data_features[:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [54]:
# check shapes

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [55]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [57]:
vocab[400:500]

['ap male athlete',
 'apg',
 'appear',
 'appearance',
 'aren',
 'arguably',
 'arguably ol',
 'arguably ol darnell',
 'argument',
 'arians',
 'arians asked',
 'ask',
 'asked',
 'asked drew',
 'asked drew lock',
 'asking',
 'ass',
 'assist',
 'assistant',
 'assistant special',
 'assistant special teams',
 'assists',
 'assists rebounds',
 'assists steals',
 'assuming',
 'asts',
 'athlete',
 'athletes',
 'athletes decade',
 'atlanta',
 'atlanta hawks',
 'atlanta hawks 27',
 'attempt',
 'attempts',
 'attempts game',
 'auman',
 'austin',
 'available',
 'average',
 'average 200',
 'averaged',
 'averaged 20',
 'averaging',
 'awards',
 'aware',
 'away',
 'away making',
 'away making playoffs',
 'awkwardly',
 'azcardinals',
 'backs',
 'backup',
 'backup season',
 'bad',
 'baker',
 'baker mayfield',
 'baldwin',
 'ball',
 'baltimore',
 'baltimore ravens',
 'bam',
 'bam adebayo',
 'barkley',
 'barrett',
 'barring',
 'based',
 'basically',
 'basket',
 'basketball',
 'basketball reference',
 'bay',
 

## MODELING

### Logistic Regression

In [61]:
from sklearn.linear_model import LogisticRegression

In [62]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [63]:
# shape check

train_data_features.shape, y_train.shape

((1296, 5000), (1296,))

In [64]:
lr.fit(train_data_features, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
lr.score(train_data_features, y_train)

0.9953703703703703

In [66]:
lr.score(test_data_features, y_test)

0.9560185185185185

### Feature comparison

Creates a dataframe that matches features to coefficients

In [67]:
coef_list = lr.coef_.tolist()

In [68]:
coef_list = coef_list[0]

In [69]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [70]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
2583,nba,-2.501418
1419,heat,-1.324256
1383,harden,-1.233706
4845,warriors,-1.204540
1790,lebron,-1.130271
1678,kat,-1.087548
2011,luka,-1.055085
2697,okc,-0.942138
4538,suns,-0.901788
1742,lakers,-0.834941


### Let's throw out these unfair words and rerun

In [71]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [72]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [73]:
lr.fit(train_data_features, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
lr.score(train_data_features, y_train)

0.9915123456790124

In [75]:
lr.score(test_data_features, y_test)

0.9421296296296297

In [76]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1490,heat,-1.405777
4825,warriors,-1.238436
1456,harden,-1.180648
1879,lebron,-1.131791
2092,luka,-1.101842
1749,kat,-1.066598
2710,okc,-0.956601
2934,points,-0.909744
1776,kings,-0.870408
4674,tonight,-0.856530


### Decision Tree

In [77]:
from sklearn.tree import DecisionTreeClassifier

In [78]:
tree = DecisionTreeClassifier()

In [79]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [80]:
tree.score(train_data_features, y_train)

0.9992283950617284

In [81]:
tree.score(test_data_features, y_test)

0.7893518518518519

### Random Forest

In [82]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
forest = RandomForestClassifier(n_estimators=1000)

In [88]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [89]:
forest.score(train_data_features, y_train)

0.9992283950617284

In [90]:
forest.score(test_data_features, y_test)

0.8888888888888888

###  Matrix on Logistic Regression

In [91]:
from sklearn.metrics import confusion_matrix

In [92]:
y_pred = lr.predict(test_data_features)

In [93]:
cm = confusion_matrix(y_test, y_pred)

In [94]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [95]:
cm_df

,predict_neg,predict_pos
actual_neg,189,8
actual_pos,17,218


## Checking where our model failed

In [96]:
comparison_df = pd.DataFrame({'y_actual': y_test,
                              'y_predicted': y_pred})

In [97]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [98]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [99]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [100]:
mismatches

,y_actual,y_predicted,title
t3_egvgl9,nba,nfl,charania the nuggets has been open to discussing deals around young talents malik beasley and juancho hernangomez but have asked for a high value ahead of the february trade deadline according to sources both turned down extensions this past summer
t3_ehboan,nfl,nba,highlight tavon wilson called for unnecessary roughness
t3_ehc6su,nba,nfl,aside from the 2015 16 season what is the best nba season in nba history
t3_ehgzu1,nfl,nba,siciliano so game 256 comes down to one play
t3_ehiiue,nfl,nba,nfl draft order top 20 set
t3_ehlj3x,nfl,nba,which playoff game s will romo cast
t3_ehm1xy,nfl,nba,fans of the 8 teams playing this weekend how does everyone feel about their first round matchup
t3_ehnm3w,nfl,nba,jackson xavien howard arrest details
t3_ehnwvg,nfl,nba,person tre boston on cam newton the rest of those boys are playing to 40 but they got o lines why does cam get to have just average lines get beat up amp the rest of these guys have healthy o lines and get the same treatment as aw man these guys can play til they re 40 with ease
t3_ehpy1s,nba,nfl,stupid question why did the lakers sign cousins to a one year contract


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [103]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=stopwords,
                            max_features=5000,
                            ngram_range=(1, 3))

In [104]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [105]:
lr.fit(train_data_features, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [106]:
lr.score(train_data_features, y_train)

0.9776234567901234

In [107]:
lr.score(test_data_features, y_test)

0.8958333333333334

### Let's try on some other subreddits

In [108]:
train = pd.concat([politics_test, conservative_test])

In [109]:
X = train[['title']]
y = train['subreddit']

In [110]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [112]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [113]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [114]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [115]:
lr = LogisticRegression(penalty = 'l2')

In [116]:
train_data_features.shape, y_train.shape

((1342, 5000), (1342,))

In [117]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9791356184798807

In [118]:
lr.score(test_data_features, y_test)

0.7589285714285714

In [119]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [120]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
434,christmas,-1.587226
475,cnn,-1.035872
1773,media,-1.002236
1289,judges,-0.968403
1363,left,-0.897656
2588,pelosi,-0.858753
2209,ny,-0.833268
1169,impeached,-0.823107
4054,reveals,-0.821288
1141,hunter,-0.815906


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [121]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [122]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [123]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [124]:
import requests

In [125]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [133]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [134]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [154]:
colors['yellow']

(255, 255, 20)

In [136]:
colors['red']

(229, 0, 0)

In [137]:
colors['black']

(0, 0, 0)

In [138]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [140]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1, 100], [5, 2, 100])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [141]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [142]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [143]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [145]:
distance(colors['red'], colors['baby shit brown']) > distance(colors['red'], colors['pink'])

False

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [146]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [148]:
closest(colors, colors['red'])

['red',
 'fire engine red',
 'bright red',
 'tomato red',
 'cherry red',
 'scarlet',
 'vermillion',
 'orangish red',
 'cherry',
 'lipstick red']

... or the ten colors closest to (150, 60, 150):

In [149]:
closest(colors, [150, 60, 150])

['warm purple',
 'medium purple',
 'ugly purple',
 'light eggplant',
 'purpleish',
 'purplish',
 'purply',
 'light plum',
 'purple',
 'muted purple']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [153]:
closest(colors, subtractv(colors['green'], colors['yellow']))

['black',
 'dark navy blue',
 'very dark blue',
 'dark navy',
 'midnight',
 'midnight blue',
 'almost black',
 'night blue',
 'navy blue',
 'navy']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [155]:
closest(colors, addv(colors['blue'], colors['green']))

['bright turquoise',
 'bright light blue',
 'bright aqua',
 'cyan',
 'neon blue',
 'aqua blue',
 'bright cyan',
 'bright sky blue',
 'aqua',
 'bright teal']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [156]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'purple grey',
 'steel grey',
 'battleship grey',
 'grey purple',
 'purplish grey',
 'greyish purple',
 'steel',
 'warm grey',
 'green grey']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [157]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['orange'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

['azure',
 'turquoise blue',
 'bright sky blue',
 'aqua blue',
 'neon blue',
 'water blue',
 'topaz',
 'cerulean',
 'aqua',
 'dark sky blue']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [158]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['true green',
 'dark grass green',
 'grassy green',
 'racing green',
 'forest',
 'bottle green',
 'dark olive green',
 'darkgreen',
 'forrest green',
 'grass green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [159]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

Roses are red, violets are blue
Roses are fire engine red, violets are vibrant blue
Roses are orangered, violets are electric blue
Roses are red orange, violets are vibrant blue
Roses are orange red, violets are strong blue
Roses are tomato, violets are primary blue
Roses are orangered, violets are ultramarine blue
Roses are red orange, violets are vivid blue
Roses are tomato red, violets are light royal blue
Roses are red orange, violets are purplish blue
Roses are bright orange, violets are blue/purple
Roses are reddish orange, violets are purplish blue
Roses are red orange, violets are purpley blue
Roses are orangered, violets are blue purple


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [160]:
import spacy
nlp = spacy.load('en_core_web_sm')

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [161]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 100.13540510543841]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [162]:
closest(colors, avg_color)

['reddish grey',
 'brownish grey',
 'brownish',
 'brown grey',
 'mocha',
 'grey brown',
 'puce',
 'dull brown',
 'pinkish brown',
 'dark taupe']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [163]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['sickly yellow',
 'piss yellow',
 'puke yellow',
 'vomit yellow',
 'dirty yellow',
 'mustard yellow',
 'dark yellow',
 'olive yellow',
 'macaroni and cheese',
 'pea']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [ ]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [175]:
dracula[566]

seems

In [176]:
dracula[566].vector

array([ 3.1053522 , -3.9924908 ,  1.7231    , -2.9991581 ,  1.3582926 ,
        1.9504764 , -3.769898  , -1.8412848 , -5.426508  , -0.7949095 ,
        1.3359009 , -2.4829035 ,  4.6697035 ,  1.8289636 ,  0.6450138 ,
       -2.3445365 , -1.422522  ,  3.7582188 , -0.36370683,  0.28704146,
        0.42762387,  0.6853549 ,  5.1715455 , -0.4891094 ,  0.50067395,
       -2.6400118 , -0.03206313, -2.3086338 ,  1.2299111 , -0.1430639 ,
       -2.742114  ,  0.04028332, -1.9261653 ,  3.103902  ,  4.660147  ,
        1.6766165 ,  3.4759216 ,  4.6845655 , -2.11274   , -0.8327563 ,
        0.39947295, -0.47227266, -3.437942  ,  0.9268136 ,  1.7516729 ,
        1.2538236 , -1.7630681 , -1.2313899 , -4.2908125 , -0.6266174 ,
       -3.5170164 ,  5.0556974 ,  3.0723155 , -2.9609756 , -0.62665784,
       -0.5082997 , -0.41967416, -0.46261868, -1.1399795 ,  0.7362545 ,
       -1.7602625 , -0.9472228 ,  4.825426  , -1.264292  , -2.2375488 ,
        2.3394156 ,  5.3166494 , -2.6943674 , -0.8293718 ,  2.45

In [177]:
# previously we've used the _sm model, which doesn't include all vectors.
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

In [178]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [179]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [183]:
nlp.vocab['dirt'].vector

array([ 5.7953e-01,  3.0641e-02, -5.5527e-01, -3.3522e-01,  2.0271e-01,
        1.7925e-01, -3.4121e-01, -2.6266e-01, -3.3045e-01,  1.3351e+00,
       -1.8667e-01,  5.8385e-02, -6.9727e-01,  5.3980e-01, -4.2912e-01,
       -4.4030e-02, -2.0955e-01,  1.0968e+00,  1.4558e-01,  7.2086e-01,
       -2.1120e-02,  1.1126e-01,  5.9386e-01,  4.0511e-01,  4.7604e-01,
        6.4190e-02, -2.6182e-01,  6.0443e-01,  1.6239e-01, -7.8519e-03,
        5.9229e-01, -8.8481e-02, -4.1253e-01,  1.6937e-01, -4.4535e-01,
        9.9473e-02,  9.9226e-02,  1.0245e-01,  6.7635e-01, -1.0198e-01,
        2.9183e-01,  4.1507e-01,  1.1258e-02,  2.2715e-01, -1.8284e-01,
       -3.1406e-01,  1.4397e-01, -2.0236e-01, -2.8882e-01,  1.3227e-01,
        1.4972e-01,  2.3291e-01,  3.2087e-01,  7.0696e-01,  2.8489e-01,
        2.5489e-01, -3.5192e-01, -1.8813e-01,  9.7779e-02,  3.5610e-01,
       -3.8677e-01,  6.7187e-01,  9.6401e-02,  5.4824e-01,  3.9444e-01,
       -3.3573e-01, -6.5211e-02, -3.0640e-01,  3.2048e-01,  1.63

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [184]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [185]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [186]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

True

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [187]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [188]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

['tennis',
 'coach',
 'game',
 'teams',
 'Junior',
 'junior',
 'Team',
 'school',
 'boys',
 'leagues']

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [189]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

['night',
 'Day',
 'day',
 'Evening',
 'evening',
 'morning',
 'Morning',
 'afternoon',
 'nights',
 'Nights']

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [ ]:
spacy_closest(tokens, vec("wine"))

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [ ]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

The closest words to "water":

In [ ]:
spacy_closest(tokens, vec("water"))

But if you add "frozen" to "water," you get "ice":

In [ ]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

You can even do analogies! For example, the words most similar to "grass":

In [ ]:
spacy_closest(tokens, vec("grass"))

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [ ]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.